In [43]:
import pandas as pd
def analyze_positions_per_game(data, game):
    """
    Analysiert, wie oft Spieler in der grünen, gelben oder roten Zone standen,
    während der andere Spieler geschlagen hat.

    Parameters:
        data: pandas.DataFrame
            Die Daten mit Spielerpositionen und Schlägen.
        game: int
            Das Spiel, das analysiert werden soll.

    Returns:
        dict:
            Ein Dictionary mit den Häufigkeiten der Positionen in den Zonen für jeden Spieler.
    """
    # Filtere die Daten für das ausgewählte Spiel
    game_data = data[data["Game"] == game]
    shots = game_data[game_data["Spieler schlägt"] == "Ja"]

    # Zonenbegrenzungen (in Metern)
    green_zone = (-2, 2)  # X-Werte der grünen Zone
    yellow_zone_left = (-3, -2)  # X-Werte der linken gelben Zone
    yellow_zone_right = (2, 3)  # X-Werte der rechten gelben Zone
    red_zone_left = (-5, -3)  # X-Werte der linken roten Zone
    red_zone_right = (3, 5)  # X-Werte der rechten roten Zone

    # Initialisiere Zähler für jede Zone pro Spieler
    zone_counts = {0: {"green": 0, "yellow": 0, "red": 0},
                   1: {"green": 0, "yellow": 0, "red": 0}}

    # Iteriere über jeden Schlag
    for _, shot in shots.iterrows():
        frame = shot["Frame"]
        object_id = shot["Object.ID"]  # Schlagender Spieler
        opponent_id = 1 - object_id    # Gegenspieler

        # Position des Gegenspielers im aktuellen Frame
        opponent_data = game_data[(game_data["Frame"] == frame) & (game_data["Object.ID"] == opponent_id)]

        if opponent_data.empty:
            continue  # Überspringe Frames ohne gültige Daten

        opponent_x = opponent_data["Transformed.X"].iloc[0]

        # Überprüfe, in welcher Zone der Gegner steht
        if green_zone[0] <= opponent_x <= green_zone[1]:
            zone_counts[opponent_id]["green"] += 1
        elif yellow_zone_left[0] <= opponent_x <= yellow_zone_left[1] or yellow_zone_right[0] <= opponent_x <= yellow_zone_right[1]:
            zone_counts[opponent_id]["yellow"] += 1
        elif red_zone_left[0] <= opponent_x <= red_zone_left[1] or red_zone_right[0] <= opponent_x <= red_zone_right[1]:
            zone_counts[opponent_id]["red"] += 1

    return zone_counts

data = pd.read_csv("csv.csv")

zone_analysis = analyze_positions_per_game(data, game="3")
print(zone_analysis)


{0: {'green': 8, 'yellow': 2, 'red': 1}, 1: {'green': 8, 'yellow': 1, 'red': 1}}


In [35]:
def count_zone_visits(data, game, object_id, zone):
    """
    Zählt die Aufenthalte eines Spielers in einer bestimmten Zone, wenn der andere Spieler geschlagen hat.
    
    Args:
        data: DataFrame mit Spieldaten
        game: Spielnummer
        object_id: ID des Spielers, dessen Aufenthalte gezählt werden
        zone: Zielzone ("grün", "gelb" oder "rot")
        
    Returns:
        Anzahl der Aufenthalte in der angegebenen Zone
    """
    # Filtere Daten für das angegebene Spiel
    game_data = data[(data["Game"] == game) & (data["Spieler schlägt"] == "Ja") & (data["Object.ID"] == 1- object_id)]

    # Definiere die Zonen
    zone_limits = {
        "grün": (-2, 2),
        "gelb_links": (-3, -2),
        "gelb_rechts": (2, 3),
        "rot_links": (-5, -3),
        "rot_rechts": (3, 5)
    }
    
    # Wähle die passende Zonen-Grenze basierend auf der Eingabe
    if zone == "grün":
        limits = zone_limits["grün"]
    elif zone == "gelb":
        limits = zone_limits["gelb_links"], zone_limits["gelb_rechts"]
    elif zone == "rot":
        limits = zone_limits["rot_links"], zone_limits["rot_rechts"]
    else:
        return 0  # Ungültige Zone
    
    # Zähle Aufenthalte des Spielers in der Zone
    count = 0
    for _, shot in game_data.iterrows():
        frame = shot["Frame"]
        player_data = data[(data["Frame"] == frame) & (data["Object.ID"] == object_id)]
        
        if not player_data.empty:
            player_x = player_data["Transformed.X"].iloc[0]
            
            if zone == "grün":
                if limits[0] <= player_x <= limits[1]:
                    count += 1
            else:
                if limits[0][0] <= player_x <= limits[0][1] or limits[1][0] <= player_x <= limits[1][1]:
                    count += 1
    
    return count


In [34]:
game_data = data[(data["Game"] == "1") & (data["Spieler schlägt"] == "Ja") & (data["Object.ID"] == 1)]

In [42]:
result = count_zone_visits(data, game="1", object_id=0, zone="rot")
print(f"Spieler 1 war {result} Mal in der grünen Zone.")


Spieler 1 war 0 Mal in der grünen Zone.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as patches
import math
import numpy as np

In [79]:
data = pd.read_csv("csv.csv")
game_data = data[(data["Game"] == "2") & (data["Spieler schlägt"] == "Ja") & (data["Object.ID"] == 1)]

In [40]:
A= (2.4193092395990625, -11.321411218413012)
B= (-7.5, -11.89)
C= (8.23 / 2, -11.89)
vector_AB = (B[0] - A[0], B[1] - A[1])
vector_AC = (C[0] - A[0], C[1] - A[1])

length_AB = math.sqrt(vector_AB[0]**2 + vector_AB[1]**2)
length_AC = math.sqrt(vector_AC[0]**2 + vector_AC[1]**2)

vector_AB_normalized = (vector_AB[0] / length_AB, vector_AB[1] / length_AB)
vector_AC_normalized = (vector_AC[0] / length_AC, vector_AC[1] / length_AC)

vector_bisector = (vector_AB_normalized[0] + vector_AC_normalized[0], vector_AB_normalized[1] + vector_AC_normalized[1])

    # Schnittpunkt D berechnen
t = ((C[1] - B[1]) * (B[0] - A[0]) - (C[0] - B[0]) * (B[1] - A[1])) / ((C[1] - B[1]) * vector_bisector[0] - (C[0] - B[0]) * vector_bisector[1])
Dx = A[0] + t * vector_bisector[0]
Dy = A[1] + t * vector_bisector[1]
print(Dx, Dy)

2.3431587681307775 -11.89


(-1.862221166439532, 11.223787255279438)
(-0.09870281779354251, -11.890000000000002)
(-0.5987028177935425, 0.4012971822064575)
2.4193092395990625
(1.465493182900295, 11.829724270190484)
(0.07413152352045183, -11.890000000000002)
(-0.4258684764795482, 0.5741315235204518)
-3.986006724973702
(-0.2494656195529523, 10.63545986876812)
(-0.013962170003406743, -11.89)
(-0.5139621700034067, 0.4860378299965933)
-2.5847896362775784
(-1.528494724601876, 12.094452747357463)
(-0.07568819936662119, -11.890000000000002)
(-0.5756881993666212, 0.4243118006333788)
1.4000044575394726
(3.895899075578084, 11.510888385943916)
(-1.3839380916155313, -11.889999999999999)
(-1.8839380916155313, -0.8839380916155313)
-5.202909015567945
0


In [72]:
data = pd.read_csv("csv.csv")

grün = count_zone_visits(data, game="2", object_id=1, zone="rot")
print(grün)

(-1.862221166439532, 11.223787255279438)
(-0.09870281779354251, -11.890000000000002)
2.4193092395990625


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [91]:
def count_zone_occurrences(data, game, zone, object_id):
    # Filtere Daten für das angegebene Spiel
    game_data = data[(data["Game"] == game) & (data["Spieler schlägt"] == "Ja") & (data["Object.ID"] == object_id)]

    count = 0

    for _, shot in game_data.iterrows():
        frame = shot["Frame"]
        player_data = data[(data["Frame"] == frame) & (data["Object.ID"] == 1 - object_id)]
        
        A = shot["Transformed.X"], shot["Transformed.Y"]

        if -2 <= A[0] <= 2:
            field_width = 10.97
            if object_id == 0:
                B, C, X, Z = (-field_width / 2, 11.89), (field_width / 2, 11.89), 11.89, 1.5
            else:
                B, C, X, Z = (-field_width / 2, -11.89), (field_width / 2, -11.89), -11.89, -1.5

        elif A[0] < -2:
            field_width = 8.23
            if object_id == 0:
                B, C, X, Z = (-field_width / 2, 11.89), (7.5, 11.89), 11.89, 1.5
            else:
                B, C, X, Z = (-field_width / 2, -11.89), (7.5, -11.89), -11.89, -1.5

        else:
            field_width = 8.23
            if object_id == 0:
                B, C, X, Z = (-7.5, 11.89), (field_width / 2, 11.89), 11.89, 1.5
            else:
                B, C, X, Z = (-7.5, -11.89), (field_width / 2, -11.89), -11.89, -1.5

        D = calculate_intersection_point(A, B, C)

        zone_limits = {
            "grün": (D[0] - 0.5, D[0] + 0.5),
            "gelb_links": (D[0] - 0.5, D[0] - 2),
            "gelb_rechts": (D[0] + 0.5, D[0] + 2),
            "rot_links": (D[0] - 2, D[0] - 5.5),
            "rot_rechts": (D[0] + 2, D[0] + 5.5)
        }

        if zone == "grün":
            limits = zone_limits[zone]
        elif zone == "gelb":
            limits = zone_limits["gelb_links"], zone_limits["gelb_rechts"]
        elif zone == "rot":
            limits = zone_limits["rot_links"], zone_limits["rot_rechts"]
        else:
            continue

        if not player_data.empty:
            player_x = player_data["Transformed.X"].iloc[0]

            if zone == "grün":
                if limits[0] <= player_x <= limits[1]:
                    count += 1
            else:
                limits = tuple(sorted(l) for l in limits)
                if (limits[0][0] <= player_x <= limits[0][1] or limits[1][0] <= player_x <= limits[1][1]):
                    count += 1

    return count


In [96]:
test = count_zone_occurrences(data, game="2", zone="grün", object_id=0)
print(test)

2


In [99]:
data = pd.read_csv("csv.csv")
# Daten filtern
filtered_data = data[(data['Game'] == "2") & 
                     (data['Object.ID'] == 0) & 
                     (data['Speed'].isna())]

print(filtered_data)


Empty DataFrame
Columns: [Frame, Object.ID, Transformed.X, Transformed.Y, Spiel läuft, Game, Speed, Spieler schlägt]
Index: []
